# 🛡️ AI Governance Autopilot (Snowflake + Cortex)

This Snowflake Notebook demonstrates how to run the **AI Governance Autopilot** inside your Snowflake environment.

It will:
- Connect to the active Snowflake session
- Use Cortex to classify column sensitivity
- Apply `DATA_SENSITIVITY` tags
- Attach `PII_ROW_POLICY` when PII is detected
- Log all actions to `GOVERNANCE_AUTOPILOT_LOG`


## 1️⃣ Connect to Snowflake & Inspect Context

This cell uses `get_active_session()` which is available inside **Snowflake Notebooks**.

In [ ]:
from snowflake.snowpark.context import get_active_session
import snowflake.cortex as cortex

session = get_active_session()
print(f"Current database: {session.get_current_database()}")
print(f"Current schema: {session.get_current_schema()}")

## 2️⃣ Ensure Governance Objects Exist

Before running the Autopilot, make sure you've executed the SQL scripts from the repository:

- `sql/01_setup_tags_and_log.sql` – creates the `DATA_SENSITIVITY` tag and `GOVERNANCE_AUTOPILOT_LOG` table
- `sql/02_row_access_policy.sql` – creates the `PII_ROW_POLICY` row access policy
- `sql/03_alerts_and_tasks.sql` – optional: creates a simulated Slack notifier and task for untagged tables

Once those scripts have been run (in Worksheets or here), continue below.

## 3️⃣ Define the Governance Autopilot Functions

This cell defines the core logic for:
- Cortex-based sensitivity classification
- Automated tagging
- Optional row policy enforcement
- Governance audit logging


In [ ]:
from snowflake.snowpark import Session

def classify_column_sensitivity(col_name: str, values, model: str = "llama3.1-8b") -> str:
    """Classify a column's sensitivity using a Cortex model.

    Returns one of: PII, CONFIDENTIAL, INTERNAL, PUBLIC
    """
    # Limit values in the prompt for safety / size
    sample_values = list(values)[:5]

    prompt = f"""Classify this column's sensitivity.
Column: {col_name}
Values: {sample_values}
Labels: PII, CONFIDENTIAL, INTERNAL, PUBLIC
Return only the label."""

    result = cortex.complete(model=model, prompt=prompt, session=session)
    label = str(result).strip().upper()
    allowed = {"PII", "CONFIDENTIAL", "INTERNAL", "PUBLIC"}
    return label if label in allowed else "INTERNAL"


def classify_and_protect_table(full_table_name: str) -> None:
    """Run the full governance Autopilot on a Snowflake table.

    Steps:
      1. Sample rows from the table
      2. Use Cortex to classify each column's sensitivity
      3. Apply DATA_SENSITIVITY tag per column
      4. Attach PII_ROW_POLICY if any column is PII
      5. Write all actions to GOVERNANCE_AUTOPILOT_LOG
    """
    print(f"=== Running AI Governance Autopilot for {full_table_name} ===")

    df = session.table(full_table_name)
    sample_pdf = df.limit(10).to_pandas()

    has_pii = False

    # Ensure the log table exists (safety check)
    session.sql("""CREATE TABLE IF NOT EXISTS GOVERNANCE_AUTOPILOT_LOG (
            EVENT_TIME   TIMESTAMP DEFAULT CURRENT_TIMESTAMP(),
            OBJECT_TYPE  STRING,
            OBJECT_NAME  STRING,
            COLUMN_NAME  STRING,
            ACTION       STRING,
            DETAILS      STRING
        );
    """).collect()

    for col_name in sample_pdf.columns:
        values = sample_pdf[col_name].astype(str).tolist()
        raw_label = classify_column_sensitivity(col_name, values)
        label = raw_label.upper()

        print(f"  → Cortex classified {col_name} as: {label} (raw: {raw_label})")

        # Apply DATA_SENSITIVITY tag
        tag_sql = f"""ALTER TABLE {full_table_name}
MODIFY COLUMN {col_name}
SET TAG DATA_SENSITIVITY = '{label}'"""
        session.sql(tag_sql).collect()

        log_sql = f"""INSERT INTO GOVERNANCE_AUTOPILOT_LOG
(OBJECT_TYPE, OBJECT_NAME, COLUMN_NAME, ACTION, DETAILS)
VALUES('TABLE', '{full_table_name}', '{col_name}', 'TAG_APPLIED', 'DATA_SENSITIVITY={label}')"""
        session.sql(log_sql).collect()

        if label == "PII":
            has_pii = True

    if has_pii:
        print(f"PII detected in {full_table_name} → ensuring PII_ROW_POLICY is attached...")
        try:
            policy_sql = f"""ALTER TABLE {full_table_name}
SET ROW ACCESS POLICY PII_ROW_POLICY"""
            session.sql(policy_sql).collect()

            log_policy = f"""INSERT INTO GOVERNANCE_AUTOPILOT_LOG
(OBJECT_TYPE, OBJECT_NAME, COLUMN_NAME, ACTION, DETAILS)
VALUES('TABLE', '{full_table_name}', NULL, 'POLICY_ATTACHED', 'PII_ROW_POLICY')"""
            session.sql(log_policy).collect()
        except Exception as e:
            print(f"  Table already has a ROW ACCESS POLICY or attach failed: {e}")
            log_skip = f"""INSERT INTO GOVERNANCE_AUTOPILOT_LOG
(OBJECT_TYPE, OBJECT_NAME, COLUMN_NAME, ACTION, DETAILS)
VALUES('TABLE', '{full_table_name}', NULL, 'POLICY_SKIPPED', 'ROW_ACCESS_POLICY already present or attach failed')"""
            session.sql(log_skip).collect()
    else:
        print(f"No PII detected in {full_table_name} → no row policy attached.")

    print("=== Autopilot classification complete ===")

## 4️⃣ Run the Autopilot on a Demo Table

Make sure `PUBLIC.CUSTOMERS_DEMO` exists (or adjust the table name below).

In [ ]:
# Example demo run
classify_and_protect_table("PUBLIC.CUSTOMERS_DEMO")

## 5️⃣ Inspect Tags and Governance Log

Query `INFORMATION_SCHEMA` to see the `DATA_SENSITIVITY` tags, and query `GOVERNANCE_AUTOPILOT_LOG` for the audit trail.

In [ ]:
# View sensitivity tags (adjust database/schema if needed)
session.sql("""
SELECT *
FROM TABLE(
  INFORMATION_SCHEMA.TAG_REFERENCES_ALL_COLUMNS('GOVERNANCE_DB.PUBLIC.DATA_SENSITIVITY')
)
ORDER BY OBJECT_NAME, COLUMN_NAME
""").show()

# View recent governance log entries
session.sql("""
SELECT *
FROM GOVERNANCE_AUTOPILOT_LOG
ORDER BY EVENT_TIME DESC
LIMIT 50
""").show()